# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:


# Details Student 2:

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

Counting the number of males and females in our training data.

In [11]:
count_m=(df_train['gender'] == 'm').sum()
count_f=df_train.shape[0]-count_m
print(f'Number of male:{count_m}')
print(f'Number of female:{count_f}')

Number of male:575
Number of female:178


Let's start by cleaning the text, which includes non-Hebrew words, punctuation and multiple leading/trailings spaces.

In [12]:
def clean_text(text):
    
    cleaned_text = re.sub(r'[^\u0590-\u05FF]+', ' ', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text=cleaned_text.strip()
    
    return cleaned_text


df_train['story'] = df_train['story'].apply(clean_text)

Splitting the df_train into X_train, representing the stories, and y_train, representing the gender of each story.

In [13]:
df_train['gender']=np.where(df_train['gender'] == 'm',1,0)
X_train=df_train['story']
y_train=df_train['gender']
df_train.loc[0:4]

,story,gender
0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,1
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,1
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,0
3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,1
4,הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכים...,0


Selecting the type of vectorizer to apply to the text stories and returning the resulting vectorized representation.

In [14]:
def vectorizer(X_train,kind,X_test = None):
    
    if kind == 'Tfidf':
        vec=TfidfVectorizer(max_df=0.8,max_features=10000,ngram_range=(1,1),norm='l2')
    if kind == 'Count':
        vec=CountVectorizer(max_df=0.8,max_features=10000,ngram_range=(1,1))
        
    X_train_vec=vec.fit_transform(X_train)
    
    if X_test is not None:
        X_test_vec=vec.transform(X_test)
        return X_test_vec
    
    return X_train_vec  

In [15]:
X_train_tfidf=vectorizer(X_train,'Tfidf')
X_train_count=vectorizer(X_train,'Count')

Using gridsearch to find the optimal hyperparameters for each model.

In [16]:
def find_best_params(clf,params,X_train,y_train):
    
    
    grid_search=GridSearchCV(clf,params,cv=10)
    grid_search.fit(X_train,y_train)
    best_params=grid_search.best_params_
    print("Best parameters", best_params)
    
    return best_params

Dictionary containing hyperparameter options for each machine learning model.

In [17]:
models_params = {
    'knn': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'algorithm': ['ball_tree', 'kd_tree']
    },
    
    'dt': {
       'max_depth': [2, 4, 6, 8],
       'min_samples_split': [2, 3, 4],
       'min_samples_leaf': [1, 2, 3]
        
    },
    
    'mn':{
        'alpha': [1.0, 0.1, 0.01, 0.001],
        'fit_prior': [True, False]
        
    },
    
    'sgd':{
        'loss': ['hinge', 'log', 'modified_huber'],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'learning_rate': ['constant', 'optimal', 'invscaling'],
    },
    
    'svc':{
         'C': [0.01, 0.1, 1, 10],
         'loss': ['hinge', 'squared_hinge'],
         'penalty': ['l1', 'l2'],
    }
    
}

Training and evalutate the specified model using cross-validation with 10 folds and then returning the mean of the F1 scores obtained from the cross-validation process.

In [18]:
from sklearn.metrics import make_scorer

In [19]:
def train_and_evaluate_model(model_name,X_train,y_train):
    
    if model_name == 'knn':
        model=KNeighborsClassifier()
    elif model_name == 'dt':
        model=DecisionTreeClassifier()
    elif model_name == 'mn':
        model= MultinomialNB()
    elif model_name == 'sgd':
        model=SGDClassifier()
    elif model_name == 'svc':
        model=LinearSVC()
        
        
    best_model_params=find_best_params(model,models_params[model_name],X_train,y_train)
    model.set_params(**best_model_params)
    model.fit(X_train,y_train)
    f1_scores = cross_val_score(model, X_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=10)
    avg_f1_score = f1_scores.mean()
    print("Average F1 Score:", avg_f1_score)

We will utilize the Tfidf/Count X_train for each classifer and observe which model produces the best result.

## KNN

In [20]:
train_and_evaluate_model('knn', X_train_tfidf, y_train)
train_and_evaluate_model('knn', X_train_count, y_train)

Best parameters {'algorithm': 'ball_tree', 'n_neighbors': 9, 'weights': 'distance'}
Average F1 Score: 0.5519995139139076
Best parameters {'algorithm': 'ball_tree', 'n_neighbors': 7, 'weights': 'distance'}
Average F1 Score: 0.5496077317432462


## Decision Tree

In [21]:
train_and_evaluate_model('dt', X_train_tfidf, y_train)
train_and_evaluate_model('dt', X_train_count, y_train)

Best parameters {'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 3}
Average F1 Score: 0.584193772724161
Best parameters {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 3}
Average F1 Score: 0.5546811512532176


## Multinomial Naive Bayes

In [22]:
train_and_evaluate_model('mn', X_train_tfidf, y_train)
train_and_evaluate_model('mn', X_train_count, y_train)

Best parameters {'alpha': 0.01, 'fit_prior': False}
Average F1 Score: 0.622805331202214
Best parameters {'alpha': 0.1, 'fit_prior': True}
Average F1 Score: 0.6202094971326997


## SGD

In [23]:
train_and_evaluate_model('sgd', X_train_tfidf, y_train)
train_and_evaluate_model('sgd', X_train_count, y_train)

Best parameters {'learning_rate': 'optimal', 'loss': 'hinge', 'penalty': 'l2'}
Average F1 Score: 0.6575830382307511
Best parameters {'learning_rate': 'optimal', 'loss': 'modified_huber', 'penalty': 'elasticnet'}
Average F1 Score: 0.6613334774128967


## LinearSVC

In [24]:
train_and_evaluate_model('svc', X_train_tfidf, y_train)
train_and_evaluate_model('svc', X_train_count, y_train)

Best parameters {'C': 10, 'loss': 'hinge', 'penalty': 'l2'}
Average F1 Score: 0.6501027815871327
Best parameters {'C': 0.01, 'loss': 'squared_hinge', 'penalty': 'l2'}
Average F1 Score: 0.6800043966924914


The best model is LinearSVC (Support Vector Classifier) using Count Vectorizer. <br>Now we will apply the same process to df_test excluding y_test to make predictions on new data(X_test text stories).

In [25]:
df_test['story'] = df_test['story'].apply(clean_text)

In [26]:
X_test=df_test['story']

In [27]:
X_test_count=vectorizer(X_train,'Count',X_test)

In [28]:
best_model=LinearSVC()
params=find_best_params(best_model,models_params['svc'],X_train_count,y_train)
best_model=best_model.set_params(**params)

Best parameters {'C': 0.01, 'loss': 'squared_hinge', 'penalty': 'l2'}


In [29]:
best_model=best_model.fit(X_train_count,y_train)
y_pred=best_model.predict(X_test_count)
y_pred=np.where(y_pred == 1,'m','f')

Showing for each text_example the predicetd label.

In [30]:
df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': y_pred})

In [31]:
df_predicted.head(5)

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f


In [32]:
df_predicted.tail(5)

,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [33]:
df_predicted.to_csv('classification_results.csv',index=False)